# Setup

In [1]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

path['project'] : D:\projects\skin-lesion-classification
path['images'] : D:\projects\skin-lesion-classification\images
path['models'] : D:\projects\skin-lesion-classification\models
path['expository'] : D:\projects\skin-lesion-classification\expository
path['literature'] : D:\projects\skin-lesion-classification\literature
path['notebooks'] : D:\projects\skin-lesion-classification\notebooks
path['presentation'] : D:\projects\skin-lesion-classification\presentation
path['scripts'] : D:\projects\skin-lesion-classification\scripts
path['streamlit'] : D:\projects\skin-lesion-classification\streamlit


<a id='model_setup'></a>
# Model setup

In [2]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename

tvr: int = 3              # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0             # Random seed for parts of the process where randomness is called for.
keep_first: bool = False  # If False, then, for each lesion, we choose a random image to assign to our training set.
stratified: bool = True   # If True, we stratify classes so that the proportions remain as stable as possible after train/val split.
                          # If False, the proportions will be roughly similar.

to_classify: Union[list, dict] = ["mel",   # These are the lesion types we are interested in classifying.
                                  "bcc",   # Any missing ones will be grouped together as the 0-label class: no need to write "other" here.
                                  "akiec", # If 'other' is not desired, use restrict_to attribute above
                                  "nv",]   # Can also be a dictionary, like { 'malignant' : ['mel', 'bcc'], 'benign' : ['nv', 'bkl']}

train_one_img_per_lesion: Union[None, bool] = False

val_expansion_factor: Union[None,int] = 3

sample_size: Union[None, dict] = {"mel": 2000,     # Handling class imbalance by upsampling minority classes/downsampling majority classes
                                  "bcc": 2000,     # Specify how many images of each lesion diagnosis we want in our training set.
                                  "akiec": 2000,
                                  "nv": 2000,
                                  "other" : 2000,} # Could also leave out "other" here, and include e.g. "df: 2000" if we wanted to.

In [3]:
# Create an instance of the process class with attribute values as above.
balance_ta = process(data_dir=data_dir,
                  csv_filename=csv_filename,
                  tvr=tvr,
                  seed=seed,
                  keep_first=keep_first,
                  stratified=stratified,
                  to_classify=to_classify,
                  train_one_img_per_lesion=train_one_img_per_lesion,
                  val_expansion_factor=val_expansion_factor,
                  sample_size=sample_size,)

- Loaded file 'D:\projects\skin-lesion-classification\images\metadata.csv'.
- Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
- Inserted 'label' column in dataframe, to the right of 'dx' column: 
  {'bkl': 0, 'df': 0, 'vasc': 0, 'akiec': 1, 'bcc': 2, 'mel': 3, 'nv': 4}
- Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
- Basic, overall dataframe (pre-train/test split): self.df
- Balancing classes in training set.
- Balanced training set (uses as many different images per lesion as possible): self.df_train
- Expanding validation set: will combine 3 predictions into one, for each lesion in val set.
- Expanded validation set (one image per lesion, repeated 3 times): self.df_val1
- Expanded validation set (use up to 3 different images per lesion, if available): self.df_val_a
- Small sample dataframes for code testing: self._df_train_code_test, self._df_val1_code_test, self._df_val_a_code_test


In [4]:
import torchvision.transforms as transforms
transform = transforms.Compose([
transforms.RandomCrop((300, 300)),
# transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

In [5]:
import pandas as pd
from typing import Union, List, Callable
import torchvision.models as models

source: Union[process, pd.DataFrame] = balance_ta        # Processed data to be fed into model for training.
                                                      # Must either be an instance of the process class, or a dataframe of the same format as source.df if source were an instance of the process class.
model_dir: Path = path["models"]                      # Path to directory where models/model info/model results are stored.

transform: Union[None,
                 transforms.Compose,
                 List[Callable]] = transform     # Transform to be applied to images before feeding into neural network.

filename_stem: Union[None, str] = "rn18"         # For saving model and related files. Default "rn18" (if ResNet model) or "EffNet" (if EfficientNet), or "cnn".
filename_suffix: Union[None, str] = "rndcrop" # Something descriptive and unique for future reference. Default empty string "".

# model: Union[None, models.ResNet, models.EfficientNet] = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT) # Pre-trained model. Default: ResNet18.
model: Union[None, models.ResNet, models.EfficientNet] = models.resnet18(weights="ResNet18_Weights.DEFAULT")
unfreeze_last: Union[None, bool] = True
# overwrite: Union[None, bool] = True

In [6]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import cnn

rn18_balance_ta = cnn(source=source,
                    model_dir=model_dir,
                    transform=transform,
                    filename_stem=filename_stem,
                    filename_suffix=filename_suffix,
                    model=model,
                    unfreeze_last=unfreeze_last,)

New files will be created. 
Base filename: rn18_ta_bal_uflast_10e_rndcrop_01
Attributes saved to file: D:\projects\skin-lesion-classification\models\rn18_ta_bal_uflast_10e_rndcrop_01_attributes.json


In [7]:
# import time

# tic = time.time()
# rn18_balance_ta.train()
# toc = time.time()
# print(f"Elapsed time: {toc - tic}")

Batch: 0, running loss: 1.9465781450271606
Batch: 10, running loss: 17.1575368642807
Batch: 20, running loss: 31.424762725830078
Batch: 30, running loss: 43.69475656747818
Batch: 40, running loss: 56.3113357424736
Batch: 50, running loss: 67.72473001480103
Batch: 60, running loss: 78.468190908432
Batch: 70, running loss: 88.24344402551651
Batch: 80, running loss: 98.05416315793991
Batch: 90, running loss: 107.93140667676926
Batch: 100, running loss: 118.76514822244644
Batch: 110, running loss: 129.4298613667488
Batch: 120, running loss: 140.29228848218918
Batch: 130, running loss: 150.32756447792053
Batch: 140, running loss: 160.5912516117096
Batch: 150, running loss: 169.9141914844513
Batch: 160, running loss: 179.27331912517548
Batch: 170, running loss: 189.30784785747528
Batch: 180, running loss: 198.5714014172554
Batch: 190, running loss: 207.40415370464325
Batch: 200, running loss: 215.76420772075653
Batch: 210, running loss: 224.59464305639267
Batch: 220, running loss: 233.656287

In [ ]:
# import torch
# import torch.nn as nn

# instance = rn18_balance_ta

# model = models.resnet18()
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(instance.label_codes))

# file_path_pth = instance.model_dir.joinpath(instance._filename + ".pth")
# state_dict = torch.load(file_path_pth)
# model.load_state_dict(state_dict)

# # model = models.efficientnet_b0()
# # num_ftrs = model.classifier[1].in_features
# # model.classifier[1] = nn.Linear(num_ftrs, len(instance.label_codes))

# instance.model = model
# instance.state_dict = state_dict

In [8]:
# # from utils import print_header
# from multiclass_models import get_probabilities

# instance = rn18_balance_ta

# tic = time.time()

# instance.df_probabilities_val1 = get_probabilities(df=instance.df_val1,
#                                                    data_dir=instance.data_dir,
#                                                    model_dir=instance.model_dir,
#                                                    model=instance.model,
#                                                    filename=instance._filename,
#                                                    label_codes=instance.label_codes,
#                                                    transform=instance.transform,
#                                                    batch_size=instance.batch_size,
#                                                    Print=False,
#                                                    save_as=instance._filename + "_val1",)

# toc = time.time()

# print(f"Elapsed time: {toc - tic}")

# instance.df_probabilities_val_a = get_probabilities(df=instance.df_val_a,
#                                                     data_dir=instance.data_dir,
#                                                     model_dir=instance.model_dir,
#                                                     model=instance.model,
#                                                     filename=instance._filename,
#                                                     label_codes=instance.label_codes,
#                                                     transform=instance.transform,
#                                                     batch_size=instance.batch_size,
#                                                     Print=False,
#                                                     save_as=instance._filename + "_val_a",)
# tic = time.time()

# print(f"Elapsed time: {tic - toc}")

Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_ta_bal_uflast_10e_rndcrop_00_val1_probabilities.csv
Elapsed time: 76.36412167549133
Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_ta_bal_uflast_10e_rndcrop_00_val_a_probabilities.csv
Elapsed time: 76.4004967212677


In [7]:
instance = rn18_balance_ta

file_path1 = instance.model_dir.joinpath("rn18_ta_bal_uflast_10e_rndcrop_00_val1_probabilities.csv")
file_path_a = instance.model_dir.joinpath("rn18_ta_bal_uflast_10e_rndcrop_00_val_a_probabilities.csv")

instance.df_probabilities_val1 = pd.read_csv(file_path1, index_col=0)
instance.df_probabilities_val_a = pd.read_csv(file_path_a, index_col=0)

In [8]:
from utils import print_header

instance = rn18_balance_ta

print_header("Validation set: one image per lesion")
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')]
display(instance.df_probabilities_val1[display_columns].head())

print_header("Validation set: all images per lesion")
display(instance.df_probabilities_val_a[display_columns].head())


VALIDATION SET: ONE IMAGE PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0025661,bkl,0.196147,0.057795,0.016602,0.724161,0.005296
1,HAM_0002730,ISIC_0025661,bkl,0.247946,0.015202,0.012981,0.716585,0.007287
2,HAM_0002730,ISIC_0025661,bkl,0.365886,0.045827,0.013421,0.569576,0.005289
3,HAM_0001466,ISIC_0027850,bkl,0.582575,0.007791,0.193693,0.185775,0.030167
4,HAM_0001466,ISIC_0027850,bkl,0.000759,0.000004,0.000032,0.999000,0.000204



VALIDATION SET: ALL IMAGES PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0026769,bkl,0.036362,9.617091e-01,0.000780,0.000977,0.000171
1,HAM_0002730,ISIC_0026769,bkl,0.035796,9.373081e-01,0.000950,0.025693,0.000254
2,HAM_0002730,ISIC_0025661,bkl,0.552345,8.390885e-03,0.031259,0.397282,0.010724
3,HAM_0001466,ISIC_0031633,bkl,0.015292,1.135414e-04,0.002000,0.981527,0.001068
4,HAM_0001466,ISIC_0027850,bkl,0.000478,5.629219e-07,0.000431,0.998866,0.000225


In [9]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_balance_ta

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = None#{ 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = None# OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.196147,0.057795,0.016602,0.724161,0.005296,3,3
1,HAM_0002730,ISIC_0025661,bkl,0.247946,0.015202,0.012981,0.716585,0.007287,3,3
2,HAM_0002730,ISIC_0025661,bkl,0.365886,0.045827,0.013421,0.569576,0.005289,3,3
3,HAM_0001466,ISIC_0027850,bkl,0.582575,0.007791,0.193693,0.185775,0.030167,0,3
4,HAM_0001466,ISIC_0027850,bkl,0.000759,0.000004,0.000032,0.999000,0.000204,3,3



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.036362,9.617091e-01,0.000780,0.000977,0.000171,1,1
1,HAM_0002730,ISIC_0026769,bkl,0.035796,9.373081e-01,0.000950,0.025693,0.000254,1,1
2,HAM_0002730,ISIC_0025661,bkl,0.552345,8.390885e-03,0.031259,0.397282,0.010724,0,1
3,HAM_0001466,ISIC_0031633,bkl,0.015292,1.135414e-04,0.002000,0.981527,0.001068,3,3
4,HAM_0001466,ISIC_0027850,bkl,0.000478,5.629219e-07,0.000431,0.998866,0.000225,3,3


In [10]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_balance_ta
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,145,17,2,45,16,225,0.644444
akiec,6,39,7,2,3,57,0.684211
bcc,8,15,46,9,4,82,0.560976
mel,19,5,1,109,20,154,0.707792
nv,69,18,12,150,"1,102","1,351",0.815692
All,247,94,68,315,"1,145","1,869",_
precision,0.587045,0.414894,0.676471,0.346032,0.962445,_,0.649045



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,153,16,1,39,16,225,0.68
akiec,6,37,9,2,3,57,0.649123
bcc,8,10,49,10,5,82,0.597561
mel,18,5,1,114,16,154,0.74026
nv,74,15,10,158,"1,094","1,351",0.809771
All,259,83,70,323,"1,134","1,869",_
precision,0.590734,0.445783,0.7,0.352941,0.964727,_,0.662832


In [11]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_balance_ta

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.771001,0.682623,0.597377,0.682623,0.602441,0.618426,0.649045,0.578273,0.933157,0.934257,0.913871



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.774211,0.695343,0.610837,0.695343,0.616478,0.632802,0.662832,0.588672,0.931156,0.937441,0.910373


In [12]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_balance_ta

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = { 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.196147,0.057795,0.016602,0.724161,0.005296,3,3
1,HAM_0002730,ISIC_0025661,bkl,0.247946,0.015202,0.012981,0.716585,0.007287,3,3
2,HAM_0002730,ISIC_0025661,bkl,0.365886,0.045827,0.013421,0.569576,0.005289,3,3
3,HAM_0001466,ISIC_0027850,bkl,0.582575,0.007791,0.193693,0.185775,0.030167,0,3
4,HAM_0001466,ISIC_0027850,bkl,0.000759,0.000004,0.000032,0.999000,0.000204,3,3



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.036362,9.617091e-01,0.000780,0.000977,0.000171,1,1
1,HAM_0002730,ISIC_0026769,bkl,0.035796,9.373081e-01,0.000950,0.025693,0.000254,1,1
2,HAM_0002730,ISIC_0025661,bkl,0.552345,8.390885e-03,0.031259,0.397282,0.010724,0,1
3,HAM_0001466,ISIC_0031633,bkl,0.015292,1.135414e-04,0.002000,0.981527,0.001068,3,3
4,HAM_0001466,ISIC_0027850,bkl,0.000478,5.629219e-07,0.000431,0.998866,0.000225,3,3


In [13]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_balance_ta
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,143,18,2,49,13,225,0.635556
akiec,6,39,7,2,3,57,0.684211
bcc,7,15,46,10,4,82,0.560976
mel,18,5,1,114,16,154,0.74026
nv,68,18,11,175,"1,079","1,351",0.798668
All,242,95,67,350,"1,115","1,869",_
precision,0.590909,0.410526,0.686567,0.325714,0.967713,_,0.645946



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,151,16,1,42,15,225,0.671111
akiec,4,39,9,2,3,57,0.684211
bcc,8,10,49,10,5,82,0.597561
mel,18,5,1,116,14,154,0.753247
nv,72,15,10,169,"1,085","1,351",0.803109
All,253,85,70,339,"1,122","1,869",_
precision,0.596838,0.458824,0.7,0.342183,0.967023,_,0.666403


In [14]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_balance_ta

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.7603,0.683934,0.596286,0.683934,0.599581,0.614102,0.645946,0.57103,0.933157,0.934257,0.913871



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.770465,0.701848,0.612974,0.701848,0.618306,0.634779,0.666403,0.586954,0.931156,0.937441,0.910373
